# Tutorial of in-sample resolution enhancement

This tutorial demonstrates in-sample resolution enhancement using a Xenium lung adenocarcinoma data E1. 

In [58]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np 
import scanpy as sc
from MicroMap import MicroMap

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


## Download data from Google Drive

We first download the required data from Google Drive, which will automatically create a local `data/` directory and store all related files.

In [6]:
import os

# Folder link
GDRIVE_URL = "https://drive.google.com/drive/folders/1jCCmvksz9r1DC-W-2mpXzNqpU9c4Ab8R"
DATA_DIR = "data"

# Create data directory if not exists
os.makedirs(DATA_DIR, exist_ok=True)

# Install gdown if needed
try:
    import gdown
except ImportError:
    !pip install -q gdown
    import gdown

# Download entire folder
gdown.download_folder(GDRIVE_URL, output=DATA_DIR, quiet=False)

print("\n Data download finished!")
print("Files in data/:", os.listdir(DATA_DIR))


Retrieving folder contents


Processing file 1ump0a4HWTqFuHKqX2SGFfa8BH-oMATEH adata_gt_E1.h5ad
Processing file 1wX2jJPV4PjSveY3zjP6328TNzpY9KmIf adata_visium_E1.h5ad
Processing file 1Gaem6GsJfNGo7nffEwRUa2k0Xg6HtlL_ adata_visium_rep2.h5ad
Processing file 1LFOSWH5xWUuBTd89Uwt-lIS3ao4dv8Uz HE_E1.png
Processing file 1y0rxIoiHMaAO7WwDgixjY9UjZHBM_4xD HE_rep2.png


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1ump0a4HWTqFuHKqX2SGFfa8BH-oMATEH
From (redirected): https://drive.google.com/uc?id=1ump0a4HWTqFuHKqX2SGFfa8BH-oMATEH&confirm=t&uuid=421ef6ae-7443-4ce8-80e5-34570b86911f
To: /data/yyyu/test/imint/tutorial/data/adata_gt_E1.h5ad
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 266M/266M [00:34<00:00, 7.83MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wX2jJPV4PjSveY3zjP6328TNzpY9KmIf
To: /data/yyyu/test/imint/tutorial/data/adata_visium_E1.h5ad
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4.34M/4.34M [00:02<00:00, 1.45MB/s]
Downloading.


✅ Data download finished!
📂 Files in data/: ['HE_rep2.png', 'adata_visium_E2.h5ad', 'adata_gt_E1.h5ad', 'adata_visium_rep2.h5ad', 'adata_visium_E1.h5ad', 'HE_E1.png']



Download completed


## Load low resolution data

Next, we load the low-resolution synthetic gene expression data, which have high-resolution ground-truth.

In [4]:
adata = sc.read('./data/adata_visium_E1.h5ad')
adata

AnnData object with n_obs × n_vars = 3496 × 289
    obs: 'pxl_col_in_fullres', 'pxl_row_in_fullres', 'n_counts'
    var: 'n_cells'
    obsm: 'spatial'

## Construct and run with MicroMap object

We then initialize a MicroMap object and compute spot-specific latent priors.

In [5]:
# construct 
micro =  MicroMap( device = 'cuda:0', out_path='in_sample')

# calculate expression latent distrobution prior for each spot
micro.cal_spot_prior(adata.X)

Image features are extracted using the pathology foundation model UNI, which may take approximately 20 minutes depending on hardware and system configuration. Users are required to download the pretrained model weights pytorch_model.bin in advance from:
https://github.com/mahmoodlab/UNI

In [7]:
# calculate image features via foundation model UNI (https://github.com/mahmoodlab/UNI)
micro.cal_token_feats(   img_path = './data/HE_E1.png',
                         UNI_path = '/data/yyyu/test/UNI/code_raw/assets/ckpts/vit_large_patch16_224.dinov2.uni_mass100k/pytorch_model.bin',
                         scale = 0.4
                         )

Creating and loading model ... 
Extracting features... 
Saving features ...
Saved to: ./features.pt


Subsequently, the main model is trained to predict high-resolution gene expression.

In [8]:
# train the backbone model
micro.train( 
             spot_coord = adata.obsm['spatial'][:,[1,0]], 
             spot_radius = 137.5,
             spot_count = adata.X,
             genes = adata.var_names.tolist()
          )

  0%|▌                                                                                                                                                                                        | 1/300 [00:03<15:29,  3.11s/it]

Epoch 1: total=15.7053, nb=7.8749, size=1.5217, smooth=1.4817, cos_gene=2.6378, cos_cell=2.6655, kl_token=3.8770


  1%|█▏                                                                                                                                                                                       | 2/300 [00:06<15:22,  3.10s/it]

Epoch 2: total=6.8964, nb=3.0361, size=0.4640, smooth=0.8903, cos_gene=1.3144, cos_cell=1.1963, kl_token=2.9072


  1%|█▊                                                                                                                                                                                       | 3/300 [00:09<15:22,  3.11s/it]

...

Epoch 298: total=2.9998, nb=1.6553, size=0.0036, smooth=0.1484, cos_gene=0.2503, cos_cell=0.2045, kl_token=1.9761


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 299/300 [15:30<00:03,  3.06s/it]

Epoch 299: total=2.9990, nb=1.6524, size=0.0042, smooth=0.1492, cos_gene=0.2508, cos_cell=0.2047, kl_token=1.9791


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [15:33<00:00,  3.11s/it]

Epoch 300: total=2.9963, nb=1.6514, size=0.0037, smooth=0.1489, cos_gene=0.2492, cos_cell=0.2033, kl_token=1.9727


In [9]:
# prediction
adata_hir = micro.predict(  genes = adata.var_names.tolist(), extract_mask=False)
adata_hir # High-resolution gene expression across the entire H&E image

AnnData object with n_obs × n_vars = 687960 × 289
    obs: 'patch_y', 'patch_x'
    obsm: 'spatial', 'latent'

## Evaluation

Finally, we load the 8 μm ground-truth data and evaluate the prediction performance using multiple quantitative metrics, including Pearson correlation coefficient (PCC), Spearman rank correlation coefficient (RCC), root mean squared error (RMSE), and structural similarity index measure (SSIM).

In [22]:
from MicroMap.evaluate import evaluation_func, normalize_count

In [16]:
adata_gt = sc.read('./data/adata_gt_E1.h5ad')
adata_gt

AnnData object with n_obs × n_vars = 212082 × 289
    obs: 'patch_y', 'patch_x', 'n_counts'
    obsm: 'patch', 'spatial'

In [39]:
genes = adata_gt.var_names
tokens = adata_gt.obs_names
adata_use = adata_hir[tokens, genes].copy()
adata_use

expr_gt = adata_gt.to_df()
expr_gt_norm = normalize_count(expr_gt)

expr_use = adata_use.to_df()
expr_use_norm = normalize_count(expr_use)

In [ ]:
patch_shape = (adata_gt.obs[['patch_x', 'patch_y']].max()[0]+1, adata_gt.obs[['patch_x', 'patch_y']].max()[1]+1)
index_gt = adata_gt.obs[['patch_x', 'patch_y']].values
index_gt = (index_gt[:,0], index_gt[:,1])

metric = evaluation_func(expr_use_norm, expr_gt_norm, genes,
                         do_spearman=True, spearman_subsample=100000, spearman_jobs=8,
                         do_ssim=True, ssim_kwargs={'index_gt':index_gt, 'patch_shape':patch_shape})

/tmp/ipykernel_3716131/813806912.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  patch_shape = (adata_gt.obs[['patch_x', 'patch_y']].max()[0]+1, adata_gt.obs[['patch_x', 'patch_y']].max()[1]+1)
/tmp/ipykernel_3716131/813806912.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  patch_shape = (adata_gt.obs[['patch_x', 'patch_y']].max()[0]+1, adata_gt.obs[['patch_x', 'patch_y']].max()[1]+1)


A NaN: False A Inf: False
B NaN: False B Inf: False
PCC     0.395892
RMSE    0.042599
RCC     0.291556
SSIM    0.830690
dtype: float32


,PCC,RMSE,RCC,SSIM
ACE,0.372203,0.051588,0.312985,0.732032
ACE2,0.492616,0.042254,0.395970,0.869865
ACKR1,0.524053,0.025574,0.242501,0.936199
ADAM17,0.401452,0.062278,0.355432,0.672426
ADAM28,0.533735,0.023897,0.376406,0.930227
...,...,...,...,...
VSIG4,0.535784,0.028951,0.262331,0.896637
VWF,0.498317,0.044065,0.376600,0.797123
WFS1,0.323130,0.088869,0.295729,0.538216
WNT2,0.162203,0.057103,0.166479,0.665252


In [55]:
moranI = adata_gt.var['moranI']
metric[moranI>0.2].median()

PCC     0.524053
RMSE    0.041185
RCC     0.355439
SSIM    0.889154
dtype: float32